## Example workbook for DIC <-> EBSD linking

In [4]:
#Load third-party packages
import numpy as np
import matplotlib.pyplot as plt

from skimage import transform as tf
from skimage import io

#Load quat, ebsd and hrdic packages from 'packages' sub-directory
from packages.quat import Quat
import packages.ebsd as ebsd
import packages.hrdic as hrdic

#Set plot behaviour
#Interactive grain selection has only been tested in osx display mode
%matplotlib osx

### Load in and display DIC map

In [ ]:
#Load in DIC map
DicFilePath = "example_data/"
#Load in BSE pattern image
DicMap = hrdic.Map(DicFilePath, "B00005.txt")
#Set crop
DicMap.setCrop(xMin=15, xMax=28, yMin=30, yMax=15)

In [ ]:
#Display max shear map
DicMap.plotMaxShear(plotColourBar=True)

### Print paramaters

In [ ]:
print("Paramater \tMin(%)\tMean(%)\tMax(%)".format(np.max(DicMap.crop(DicMap.f11))))
print("Exx: \t\t{0:.2f}\t{1:.2f}\t{2:.2f}".format(np.min(DicMap.crop(DicMap.f11)),np.mean(DicMap.crop(DicMap.f11)),np.max(DicMap.crop(DicMap.f11))))
print("Eyy: \t\t{0:.2f}\t{1:.2f}\t{2:.2f}".format(np.min(DicMap.crop(DicMap.f22)),np.mean(DicMap.crop(DicMap.f22)),np.max(DicMap.crop(DicMap.f22))))
print("Max shear: \t\t{0:.2f}\t{1:.2f}\t{2:.2f}\n".format(np.min(DicMap.crop(DicMap.max_shear)),np.mean(DicMap.crop(DicMap.max_shear)),np.max(DicMap.crop(DicMap.max_shear))))
print("# points: {0}".format(len(DicMap.crop(DicMap.max_shear)[0])*len(DicMap.crop(DicMap.max_shear)[1])))

### Effective shear strain histogram

In [ ]:
data = DicMap.crop(DicMap.max_shear)
data = [item for sublist in irr_crop for item in sublist]

f, (ax1) = plt.subplots(1)
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

counts, bins, bars = ax1.hist(data, bins=np.linspace(0,0.1,200), color='r', histtype='step')
plt.gca().set_xlabel("Effective shear strain")
plt.gca().set_ylabel("Frequency")

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('irr.csv', (center, counts), delimiter='\n')


plt.show()

### Load in EBSD map then display local misorientation

In [ ]:
#Load in EBSD map and calculate misorientation
EbsdFilePath = "example_data/Map Data 2-DIC area"
EbsdMap = ebsd.Map(EbsdFilePath, "cubic")
EbsdMap.loadSlipSystems(filepath="slip_systems/cubic.txt")

EbsdMap.binData = EbsdMap.binData[::-1]  #Rotate the map 180 degrees
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 8)    #Find boundaries
EbsdMap.findGrains(minGrainSize=10)      #Find grains
EbsdMap.calcGrainMisOri(calcAxis = True)

In [ ]:
EbsdMap.plotMisOriMap(plotGBs=True, vmin=0, vmax=4)

### Find a grain of interest (click around the map)

In [ ]:
EbsdMap.locateGrainID()

### Plot misorientation for isolated grain

In [ ]:
EbsdMap.grainList[EbsdMap.currGrainId].plotMisOri()

In [ ]:
EbsdMap.grainList[EbsdMap.currGrainId].plotMisOri(component=4)

## Link EBSD map to DIC map

### Define homologous points

In [ ]:
#Set homologous points   (x, y)
DicMap.homogPoints = np.array((
    (303, 408), (120, 392), (275, 25)
))

EbsdMap.homogPoints = np.array((
    (536, 776), (236, 790), (466, 160)
))

### Display map with homologous points

In [ ]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

### Link the maps with a shift for corrections

In [ ]:
DicMap.linkEbsdMap(EbsdMap)

### Plot max shear with boundaries

In [ ]:
DicMap.plotMaxShear(plotGBs=True)

### Detect grains in the DIC map

In [ ]:
DicMap.findGrains(minGrainSize=10)

### Locate a grain of interest (click around)

In [ ]:
DicMap.locateGrainID(displaySelected=True)
#displaySelected will show the selcted grain in a separate figure window. 
#Do not close the window and it will update as you click grains.

### Plot max shear for isolated grain

In [ ]:
DicMap.grainList[DicMap.currGrainId].plotMaxShear()

### Plot a histogram of it

In [ ]:
plt.figure()
plt.hist(DicMap.grainList[DicMap.currGrainId].maxShearList, bins=100);

### Plot misorientation for isolated grain

In [ ]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
EbsdMap.grainList[ebsdGrainId].plotMisOri()

### Plot max shear for isolated grain with slip traces

In [ ]:
DicMap.grainList[DicMap.currGrainId].calcSlipTraces()
DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

### Plot boundaries without crop to see rotation

In [ ]:
plt.figure()

warpedBoundaries = tf.warp(-DicMap.ebsdMap.boundaries.astype(float), DicMap.ebsdTransform) > 0.1

warpedBoundaries = -warpedBoundaries.astype(int)

plt.imshow(warpedBoundaries, cmap="gray")